In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.

In [2]:
!nvidia-smi

Wed Apr 20 20:36:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    41W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.utils as utils
import gzip
!pip install wandb
import os
import wandb
! pip install torchsummaryX
from torchsummaryX import summary
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

checkpoints_store_path = "/content/drive/MyDrive/02710_proj/store_checkpoints"
create_folder(checkpoints_store_path)

batch_size = 1
epochs = 10
lr = 1e-3

model_id = f"Weighted_weakRM"
data_path = checkpoints_store_path + model_id
create_folder(data_path)


wandb.init(project="02710", entity="saltedfish",name=model_id)
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": batch_size,
}


Test Acc,▁█▆▅
Test loss,█▆▁▂
Train Acc:,▁▇██
Train loss:,█▃▂▁
auROC,█▁▆▃
lr,▁▁▁▁
Test Acc,87.18
Test loss,0.13607
Train Acc:,86.31
Train loss:,0.3204
auROC,0.79519


In [8]:
a_file = gzip.open("/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/training_sample_0/sequences.fa.gz", "rb")
contents = a_file.readlines()

for i in range(10):
    print(contents[i])

b'> chr1,+,866159,866259; class:0\n'
b'GCCTCGGTCCTGTTCTTCCTGATGCGTGTCTGCTGAGGCCAGGAGCTGGCTTTGGCCCAT\n'
b'GGGCCTGTCCTAGTGGGAGGCCCCAGCATGTTGAGCCAGTA\n'
b'> chr1,+,870491,870591; class:0\n'
b'GGATTGGGCTGAATTAGCAAGAAGAGGAGAAATGAGGGAAGAAAAGAGTTAAATGCATGT\n'
b'TGATTCCAAGCCCCCGCCTGCCGGGGGGACAGCGGGAGGTT\n'
b'> chr1,+,949147,949247; class:0\n'
b'AGGTAAAGGGAGGCCACGGGATGGCGGTGGGCAGCTGGCCTTCTAGTAACGAGCCCTCAG\n'
b'TGCCTTCTGTGCCTGGGGTCCCTGCCGGCGGGATGTAGAGG\n'
b'> chr1,+,949360,949460; class:0\n'


In [9]:
def read_file_helper(file_path):
    res = []
    with gzip.open(file_path, "rb") as f:
        for line in f.readlines():
            line = str(line)[2:-3]
            
            if line.startswith(">"):
                indicator = 0
                tmp = []
                loc, y = line.strip().split(";")
                chr_num, sign, start, end = loc[2:].split(",")
                tmp.extend([chr_num, sign, int(start), int(end), y[-1]])
            
            else:
                indicator += 1
                tmp.append(line)
                if indicator == 2:
                    res.append(tmp)
    df = pd.DataFrame(res, columns = ["chr_num", "sign", "start", "end", "y", "seq_part1", "seq_part2"])
    df["seq"] = df["seq_part1"] + df["seq_part2"]
    df["y"] = df["y"].astype(int)
    return df
                

def embed(sequence, instance_len, instance_stride):
    instance_num = int((len(sequence) - instance_len) / instance_stride) + 1
    bag = []
    for i in range(instance_num):
        instance = sequence[i * instance_stride:i * instance_stride + instance_len]
        instance = one_hot_encode(instance)
        bag.append(instance)
    bag = np.stack(bag).astype(float)
    return bag

def one_hot_encode(seq):
    mapping = dict(zip("ACGTN", range(5)))
    seq2 = [mapping[i] for i in seq]
    return np.eye(5)[seq2]

def create_bag(seqs, instance_len=40, instance_stride=5):
    bags = []
    for seq in seqs:
        bags.append(embed(seq, instance_len, instance_stride)) 
        
    return np.array(bags).astype(float)

class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path):
        df = read_file_helper(data_path)
        df["seq"] = df["seq_part1"] + df["seq_part2"]
        self.X, self.Y = create_bag(df["seq"]), df["y"].to_numpy()
        
        assert len(self.X) == len(self.Y)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, item):
        return self.X[item], self.Y[item]

In [10]:
train_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/training_sample_0/sequences.fa.gz"
valid_data_path = "/content/drive/MyDrive/iDeepS-master/datasets/clip/10_PARCLIP_ELAVL1A_hg19/30000/test_sample_0/sequences.fa.gz"
batch_size = 1

train_data = LibriSamples(train_data_path)
valid_data = LibriSamples(valid_data_path)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size, shuffle=True)

y_num_0 = []
y_num_1 = []
for x, y in train_loader:
    # print(x, y)
    if y == 0:
      y_num_0.append(y)
    else:
      y_num_1.append(y)

In [66]:
print(len(y_num_0))
print(len(y_num_1))

24000
6000


In [22]:
class WeakRM(nn.Module):
    def __init__(self):
        super().__init__()

        self.embedding = nn.Sequential(
            nn.Conv1d(40, 32, kernel_size=15, padding=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.25),
            nn.Conv1d(32, 16, kernel_size=5, padding=2, stride=1),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Flatten()
        )

        self.attention_v = nn.Sequential(
            nn.Linear(32, 128),
            nn.Tanh()
        )

        self.attention_u = nn.Sequential(
            nn.Linear(32, 128),
            nn.Sigmoid()
        )

        self.attention_weights = nn.Sequential(
            nn.Linear(128, 1),
            nn.Softmax()
        )

        self.cls = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.softmax = nn.Softmax()

    def forward(self, inputs, training=True, mask=None):
        inputs = torch.squeeze(inputs, 0)

        embedding = self.embedding(inputs)  # torch.Size([13, 32])
        # print(embedding.shape)

        attention_v = self.attention_v(embedding)
        attention_u = self.attention_v(embedding)

        # print(attention_u.shape, attention_v.shape)
        gated_attention = self.attention_weights(attention_u * attention_v).permute((1, 0))

        gated_attention = self.softmax(gated_attention)  # torch.Size([1, 13])

        bag_features = torch.matmul(gated_attention, embedding)
        # print(embedding.shape, gated_attention.shape)
        bag_probability = self.cls(bag_features)

        return bag_probability, gated_attention


In [24]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score, average_precision_score

model = WeakRM().cuda()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
# criterion = nn.BCELoss(weight=torch.tensor([0.8,0.2]))
criterion = nn.BCELoss()

for epoch in range(1, epochs + 1):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    num_correct = 0
    total_loss = 0

    # training samples
    # model.train()
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.float().cuda()
        y = y.float().cuda()

        # Don't be surprised - we just wrap these two lines to make it work for FP16
        outputs,_ = model(x)

        # print(outputs)
        # print("y file is:",y.dtype)
        loss = criterion(outputs[0], y)

        outputs = torch.tensor([0 if outputs[0][0]<0.5 else 1]).cuda()

        num_correct += int((outputs == y).sum())
        total_loss += float(loss)

        train_acc = 100 * num_correct / ((i + 1) * batch_size)
        train_loss = float(total_loss / (i + 1))
        
        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(train_acc),
            loss="{:.04f}".format(train_loss),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

        # Another couple things you need for FP16.
        loss.backward()  # This is a replacement for loss.backward()
        optimizer.step()  # This is a replacement for optimizer.step()
        batch_bar.update()  # Update tqdm bar
    
    batch_bar.close()  # You need this to close the tqdm bar

    print_content ="Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f} \n".format(
    epoch,
    epochs,
    100 * num_correct / (len(train_loader) * batch_size),
    float(total_loss / len(train_loader)),
    float(optimizer.param_groups[0]['lr']))

    print(print_content)

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Eval')

    num_correct = 0
    total_loss = 0

    # eval samples
    model.eval()
    predictions =[]
    label = []
    for i, (x, y) in enumerate(valid_loader):
        x = x.float().cuda()
        y = y.float().cuda()

        with torch.no_grad():
            outputs,_ = model(x)

        outputs = torch.tensor([0 if outputs[0][0]<0.5 else 1]).cuda()
        print(outputs)

        num_correct += int((outputs == y).sum())
        total_loss += float(loss)
        
        predictions.append(outputs.detach().cpu().numpy())
        label.append(y.detach().cpu().numpy())


        dev_acc = 100 * num_correct / ((i + 1) * batch_size)
        dev_loss = float(total_loss / (i + 1))
        # print(dev_loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(dev_acc),
            loss="{:.04f}".format(dev_loss),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))

        # Another couple things you need for FP16.
        batch_bar.update()  # Update tqdm bar
    batch_bar.close()  # You need this to close the tqdm bar


    auROC = roc_auc_score(y_true=label, y_score=predictions)

    wandb.log({"Train Acc:":train_acc, "Train loss:": train_loss,"Test Acc ":dev_acc,"Test loss":dev_loss,"auROC":auROC,"lr":lr})
    print(auROC)
    print_content ="Epoch {}/{}: Eval Acc {:.04f}%, Eval auROC {:.04f}%\n".format(
    epoch,
    epochs,
    dev_acc,
    auROC)
    print(print_content)



Train:   0%|          | 0/30000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
Train:   4%|▍         | 1225/30000 [00:06<02:46, 172.31it/s, acc=79.9347%, loss=0.4255, lr=0.0010, num_correct=980]

KeyboardInterrupt: ignored

In [12]:
network = WeakRM().cuda()
for x, y in train_loader:
    # print(x.permute((0, 1, 3, 2)).shape)
    x = x.float().cuda()
    # print(x)
    output = network(x)
    print(output[0].shape, output[1].shape)
    break

torch.Size([1, 1]) torch.Size([1, 13])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
df = read_file("../iDeepS/datasets/clip/1_PARCLIP_AGO1234_hg19/30000/training_sample_0/sequences.fa.gz")
df["seq"] = df["seq_part1"] + df["seq_part2"]
a, b = create_bag(df["seq"], df["y"])


In [ ]:
a.shape

(390000, 40, 5)

In [ ]:
len(b)

390000

In [ ]:
df= read_file("../iDeepS/datasets/clip/1_PARCLIP_AGO1234_hg19/30000/training_sample_0/sequences.fa.gz")


In [ ]:
[1]*4

[1, 1, 1, 1]

In [ ]:
len(df["seq"][0])

101

In [ ]:
df = read_file("../iDeepS/datasets/clip/9_PARCLIP_ELAVL1MNASE_hg19/30000/test_sample_0/sequences.fa.gz")
df.groupby("y").count()

,chr_num,sign,start,end,seq_part1,seq_part2
y,,,,,,
0,8000,8000,8000,8000,8000,8000
1,2000,2000,2000,2000,2000,2000
